# Logistic Regression

In [9]:
# base setting
import sys
sys.path.append('../src/')

In [18]:
# libraries
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from process import texthandling as th
from tqdm import tqdm

In [11]:
# logger setting
logger = getLogger(__name__)
logdir = 'log/'
if __name__ == '__main__':
    log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s')
    handler = StreamHandler()
    handler.setLevel('INFO')
    handler.setFormatter(log_fmt)
    logger.addHandler(handler)
    
    handler = FileHandler(logdir + 'logistic_reg.ipynb.log', 'a')
    handler.setLevel(DEBUG)
    handler.setFormatter(log_fmt)
    logger.setLevel(DEBUG)
    logger.addHandler(handler)

In [12]:
# data input
datadir = '../input/'
train = pd.read_csv(datadir + 'train.csv')

In [13]:
# processing

# get dummy variables
dum_sex = pd.get_dummies(train['Sex'])
dum_emb = pd.get_dummies(train['Embarked'])

train_proc = pd.concat((train, dum_sex, dum_emb), axis = 1)
#train_proc = train_proc.drop('Sex', axis = 1)
train_proc = train_proc.drop('Embarked', axis = 1)
train_proc = train_proc.drop('female', axis = 1)
train_proc = train_proc.drop('S', axis = 1)

# check missing status
# train_proc.isnull().sum()

In [6]:
name_prefix = ['Mrs', 'Master', 'Mr', 'Miss']
name_df = th.FlagWords(train_proc['Name'], name_prefix, independent=True)
name_df['prefix'] = name_df.Mr + 2*name_df.Master + 3*name_df.Mrs + 4*name_df.Miss - 1

train_proc = pd.concat([train_proc, name_df], axis=1)
# train_proc = train_proc.drop('Name', axis=1)
train_proc.groupby('prefix').count()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,male,C,Q,Mrs,Master,Mr,Miss
prefix,,,,,,,,,,,,,,,,,,
-1,24,24,24,24,24,23,24,24,24,24,10,24,24,24,24,24,24,24
0,518,518,518,518,518,399,518,518,518,518,94,518,518,518,518,518,518,518
1,40,40,40,40,40,36,40,40,40,40,7,40,40,40,40,40,40,40
2,129,129,129,129,129,112,129,129,129,129,46,129,129,129,129,129,129,129
3,180,180,180,180,180,144,180,180,180,180,47,180,180,180,180,180,180,180


In [16]:
# Logistic Regression
lr = LogisticRegression()
x_var = train_proc[['male', 'Fare']]
y_var = train_proc.Survived
# lr.fit(x_var, y_var)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
all_params = {'C': [10**i for i in range(-1, 2)],
              'fit_intercept': [True, False],
              'penalty': ['l2', 'l1'],
              'random_state': [0]}



StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
